In [24]:
import pandas as pd
import nltk, collections
from nltk.util import ngrams
from nltk.corpus import stopwords

In [25]:
df = pd.read_csv("articles_with_text.csv")
df.head()

,Unnamed: 0,URL,Title,Source,Authors,Summary,Text,Keywords,Publish date,HTML
0,0,https://www.nytimes.com/2008/06/01/weekinrevie...,Think the Economy Is Bad? Wait Till the States...,www.nytimes.com,['Louis Uchitelle'],State and city governments have yet to shrink ...,"Struggling as we are with the housing bust, th...","states, california, workers, cut, trillion, ca...",2008-06-01 00:00:00,"<div><p class=""css-axufdj evys1bk0"">Struggling..."
1,1,https://www.nytimes.com/2008/06/01/opinion/01g...,Put a Little Science in Your Life - The New Yo...,www.nytimes.com,['Brian Greene'],"The letter began by saying that, as we’ve all ...",A COUPLE of years ago I received a letter from...,"letter, written, little, book, york, writing, ...",2008-06-01 00:00:00,"<div><p class=""css-axufdj evys1bk0"">A COUPLE o..."
2,2,https://www.washingtonpost.com/wp-dyn/content/...,Villains in the Mortgage Mess? Start at Wall S...,www.washingtonpost.com,[],"Start at Wall Street.\nIn the 1990s, after the...",Villains in the Mortgage Mess? Start at Wall S...,"subprime, going, mess, lenders, loan, didnt, b...",2008-05-30 00:00:00,<div><p>\n \n \n \n</p>\n \n<p>\n</p>\n<b>Vill...
3,3,http://www.guardian.co.uk/commentisfree/2008/j...,What I told the Pope about how to shape the ne...,www.guardian.co.uk,['Will Hutton'],"Stalin was famously dismissive of the Pope, de...","For a man aged over 80, Pope Benedict XVI walk...","told, vatican, work, capitalism, church, good,...",2008-06-01 00:00:00,"<div><p class=""dcr-1jp9ryy"">For a man aged ove..."
4,4,http://www.sciam.com/article.cfm?id=matter-ant...,Matter-Antimatter Split Hints at Physics Break...,www.sciam.com,"['Jr Minkel', 'About The Author S', 'Jr Minkel...",Early data from twin experiments at the Tevatr...,Nature may have handed scientists a new clue i...,"cp, physics, antimatter, hints, particle, brea...",NaN,<div><p>Nature may have handed scientists a ne...


In [26]:
def domain_split(domain):
    #Removes all sub domains from a netloc value
    #Takes a string, splits on first '.' and returns the string to the right. Repeats until string contains just one '.'
    domain = domain.replace(".co.uk",".com")
    while domain.count(".") > 1:
        domain = domain.split(".",1)
        domain = domain[1]
    return domain

#Get domain from 'Source' value; code doesn't work for ""
domains = []
for source in df['Source']:
    domain = domain_split(source)
    domains.append(domain)
    
df['Domain'] = domains
df.head(50)

,Unnamed: 0,URL,Title,Source,Authors,Summary,Text,Keywords,Publish date,HTML,Domain
0,0,https://www.nytimes.com/2008/06/01/weekinrevie...,Think the Economy Is Bad? Wait Till the States...,www.nytimes.com,['Louis Uchitelle'],State and city governments have yet to shrink ...,"Struggling as we are with the housing bust, th...","states, california, workers, cut, trillion, ca...",2008-06-01 00:00:00,"<div><p class=""css-axufdj evys1bk0"">Struggling...",nytimes.com
1,1,https://www.nytimes.com/2008/06/01/opinion/01g...,Put a Little Science in Your Life - The New Yo...,www.nytimes.com,['Brian Greene'],"The letter began by saying that, as we’ve all ...",A COUPLE of years ago I received a letter from...,"letter, written, little, book, york, writing, ...",2008-06-01 00:00:00,"<div><p class=""css-axufdj evys1bk0"">A COUPLE o...",nytimes.com
2,2,https://www.washingtonpost.com/wp-dyn/content/...,Villains in the Mortgage Mess? Start at Wall S...,www.washingtonpost.com,[],"Start at Wall Street.\nIn the 1990s, after the...",Villains in the Mortgage Mess? Start at Wall S...,"subprime, going, mess, lenders, loan, didnt, b...",2008-05-30 00:00:00,<div><p>\n \n \n \n</p>\n \n<p>\n</p>\n<b>Vill...,washingtonpost.com
3,3,http://www.guardian.co.uk/commentisfree/2008/j...,What I told the Pope about how to shape the ne...,www.guardian.co.uk,['Will Hutton'],"Stalin was famously dismissive of the Pope, de...","For a man aged over 80, Pope Benedict XVI walk...","told, vatican, work, capitalism, church, good,...",2008-06-01 00:00:00,"<div><p class=""dcr-1jp9ryy"">For a man aged ove...",guardian.com
4,4,http://www.sciam.com/article.cfm?id=matter-ant...,Matter-Antimatter Split Hints at Physics Break...,www.sciam.com,"['Jr Minkel', 'About The Author S', 'Jr Minkel...",Early data from twin experiments at the Tevatr...,Nature may have handed scientists a new clue i...,"cp, physics, antimatter, hints, particle, brea...",NaN,<div><p>Nature may have handed scientists a ne...,sciam.com
5,5,http://www.latimes.com/news/opinion/la-op-sche...,Indefensible spending,www.latimes.com,[],And what are we spending all this money on?\nT...,What should be the most important issue in thi...,"defense, military, dollars, billion, indefensi...",2008-06-01 07:00:00,<div><p>What should be the most important issu...,latimes.com
6,6,https://www.nakedcapitalism.com/2008/06/talebs...,"Taleb’s Harsh Assessment of Bankers, Economist...",www.nakedcapitalism.com,"['Yves Smith', 'Lambert Strether', 'Arizona Sl...","“You will never,” he says, “be able to control...",Reader Michael called to my attention a wide-r...,"talebs, economists, fed, swans, doesnt, good, ...",2008-06-01 06:06:00+00:00,<blockquote><p>Reader Michael called to my att...,nakedcapitalism.com
7,7,https://real-estate-and-urban.blogspot.com/200...,Richard's Real Estate and Urban Economics Blog,real-estate-and-urban.blogspot.com,"['Richard Green', 'View My Complete Profile']",Richard Green is a professor in the Sol Price ...,Richard Green is a professor in the Sol Price ...,"school, economics, blog, state, sol, ruminatio...",NaN,"<div><p class=""description"">Richard Green is a...",blogspot.com
8,8,https://www.nytimes.com/2008/08/01/business/ec...,Fed Fears Wage Spiral That Is Little in Evidence,www.nytimes.com,['Louis Uchitelle'],Unions that negotiated big wage increases in t...,The surges this year in oil and food prices co...,"prices, spiral, worse, evidence, fed, little, ...",2008-08-01 00:00:00,"<div><p class=""css-axufdj evys1bk0"">The surges...",nytimes.com
9,11,http://www.marginalrevolution.com/marginalrevo...,Why did the HMO revolution fail?,www.marginalrevolution.com,"['Tyler Cowen', 'Alexander Tabarrok']",Mark Thoma cites this passage from Paul Krugma...,Mark Thoma cites this passage from Paul Krugma...,"revolution, thats, rising, rise, care, hmo, si...",2008-07-31 10:24:00+00:00,"<div><p>Mark Thoma <a href=""http://economistsv...",marginalrevolution.com


In [27]:
# Count most common domain names (with sub domains listed separately)

table = pd.pivot_table(df, index=['Source'], values=['Title'], aggfunc="count")
table = table.sort_values(by='Title', ascending=False)
table.iloc[0:10]

,Title
Source,
www.nytimes.com,1071
www.voxeu.org,666
krugman.blogs.nytimes.com,542
voxeu.org,502
www.project-syndicate.org,435
conversableeconomist.blogspot.com,416
delong.typepad.com,333
www.ft.com,325
worthwhile.typepad.com,268


In [31]:
# Count most common domain names (rolling sub domains up to domain level)

table = pd.pivot_table(df, index=['Domain'], values=['Title'], aggfunc="count")
table = table.sort_values(by='Title', ascending=False)
table.to_csv("top_domains.csv")
table.iloc[0:100]

,Title
Domain,
blogspot.com,2358
nytimes.com,1906
voxeu.org,1168
typepad.com,826
ft.com,640
project-syndicate.org,435
wordpress.com,345
washingtonpost.com,251
economist.com,241


In [28]:
text = df['Text'].str.cat(sep=",")
print(len(text))

sw_nltk = stopwords.words('english')
words = [word for word in text.split() if word.lower() not in sw_nltk]
new_text = " ".join(words)
print("Old length: ", len(text))
print("New length: ", len(new_text))

# first get individual words
tokenized = new_text.split()

# and get a list of all the bi-grams
bigrams = ngrams(tokenized, 2)

# get the frequency of each bigram in our corpus
bigramFreq = collections.Counter(bigrams)

# what are the ten most popular ngrams in this Spanish corpus?
bigramFreq.most_common(10)

57225290
Old length:  57225290
New length:  42715477


[(('Federal', 'Reserve'), 4287),
 (('monetary', 'policy'), 3730),
 (('interest', 'rates'), 3490),
 (('et', 'al.'), 2938),
 (('United', 'States'), 2573),
 (('New', 'York'), 2249),
 (('central', 'bank'), 2241),
 (('interest', 'rate'), 2080),
 (('Reserve', 'Bank'), 1523),
 (('central', 'banks'), 1411)]